# Obtaining the Data

This Jupyter Notebook runs through the process of obtaining the data of the 45 M&A transactions that are studied in the Dissertation. 
The structure is the following:

1. Obtain the List of 45 M&A transactions
2. Obtain and clean the data for the Banks in the sample
3. Create the matrices for the DSBM and SBM Model.

In [78]:
import numpy as np
import pandas as pd

## 1. Obtaining the List of the 45 M&A transactions

First a list of all M&A activitiesbetween 2014 and 2017 needs to be obtained:

In [79]:
MATrans1417 = pd.read_csv("Data/MAActivities2014-2017.csv") # List of all MA transactions in the observational period
MATrans1219 = pd.read_csv("Data/MAActivities2012-2019.csv",on_bad_lines="skip") # List of all MA transactions in the +-2 year period

MATrans1417 = MATrans1417[["ACQ_CERT","ACQ_INSTNAME","OUT_CERT","OUT_INSTNAME","EFFYEAR"]]
MATrans1219 = MATrans1219[["ACQ_CERT","ACQ_INSTNAME","OUT_CERT","OUT_INSTNAME","EFFYEAR"]]

The following function matches each M&A transaction to the corresponding data. Here the combined assets of the previous year and the combined deposits of the previous year are used. Note that only the former will be used later on. The data for this is obtained from the FDIC.

In [80]:
def getMAAssetsAndDeposits(MATrans1417,deflator):
    '''This helper function returns the summary statistics for choosing M&A Deals according to the Dissertation'''
    EasyData2013 = pd.read_csv("Data/Financials_L1_formatted_2013.csv")
    EasyData2014 = pd.read_csv("Data/Financials_L1_formatted_2014.csv")
    EasyData2015 = pd.read_csv("Data/Financials_L1_formatted_2015.csv")
    EasyData2016 = pd.read_csv("Data/Financials_L1_formatted_2016.csv")

    # Number of MA Deals
    nDeals = MATrans1417.shape[0]


    # Empty Vector of the combined Assets/Deposits of the Previous Year
    AssetVec = np.zeros(shape = nDeals)
    DepositVec = np.zeros(shape = nDeals)

    for i in range(nDeals):

        # ID for Target and Acquirer
        AcqID = MATrans1417["ACQ_CERT"][i]
        TarID = MATrans1417["OUT_CERT"][i]
        Year = MATrans1417["EFFYEAR"][i]

        # Depending on Year the Data of the previous Year is obtained, added and deflated       
        if Year == 2014:
            AcqAsset = EasyData2013[EasyData2013["Cert/ID"] == AcqID]["Total Assets"].to_numpy()
            TarAsset = EasyData2013[EasyData2013["Cert/ID"] == TarID]["Total Assets"].to_numpy()

            if len(AcqAsset) == 0  or len(TarAsset) == 0:
                AssetVec[i] = None
            else:
                AssetVec[i] = (AcqAsset + TarAsset)/deflator[Year]

            AcqDep = EasyData2013[EasyData2013["Cert/ID"] == AcqID]["Total Domestic Deposits"].to_numpy()
            TarDep = EasyData2013[EasyData2013["Cert/ID"] == TarID]["Total Domestic Deposits"].to_numpy()

            if len(AcqDep) == 0  or len(TarDep) == 0:
                DepositVec[i] = None
            else:
                DepositVec[i] = (AcqDep + TarDep)/deflator[Year]

        if Year == 2015:
            AcqAsset = EasyData2014[EasyData2014["Cert/ID"] == AcqID]["Total Assets"].to_numpy()
            TarAsset = EasyData2014[EasyData2014["Cert/ID"] == TarID]["Total Assets"].to_numpy()

            if len(AcqAsset) == 0  or len(TarAsset) == 0:
                AssetVec[i] = None
            else:
                AssetVec[i] = (AcqAsset + TarAsset)/deflator[Year]

            AcqDep = EasyData2014[EasyData2014["Cert/ID"] == AcqID]["Total Domestic Deposits"].to_numpy()
            TarDep = EasyData2014[EasyData2014["Cert/ID"] == TarID]["Total Domestic Deposits"].to_numpy()

            if len(AcqDep) == 0  or len(TarDep) == 0:
                DepositVec[i] = None
            else:
                DepositVec[i] = (AcqDep + TarDep)/deflator[Year]

        if Year == 2016:
            AcqAsset = EasyData2015[EasyData2015["Cert/ID"] == AcqID]["Total Assets"].to_numpy()
            TarAsset = EasyData2015[EasyData2015["Cert/ID"] == TarID]["Total Assets"].to_numpy()

            if len(AcqAsset) == 0  or len(TarAsset) == 0:
                AssetVec[i] = None
            else:
                AssetVec[i] = (AcqAsset + TarAsset)/deflator[Year]

            AcqDep = EasyData2015[EasyData2015["Cert/ID"] == AcqID]["Total Domestic Deposits"].to_numpy()
            TarDep = EasyData2015[EasyData2015["Cert/ID"] == TarID]["Total Domestic Deposits"].to_numpy()

            if len(AcqDep) == 0  or len(TarDep) == 0:
                DepositVec[i] = None
            else:
                DepositVec[i] = (AcqDep + TarDep)/deflator[Year]

        if Year == 2017:
            AcqAsset = EasyData2016[EasyData2016["Cert/ID"] == AcqID]["Total Assets"].to_numpy()
            TarAsset = EasyData2016[EasyData2016["Cert/ID"] == TarID]["Total Assets"].to_numpy()

            if len(AcqAsset) == 0  or len(TarAsset) == 0:
                AssetVec[i] = None
            else:
                AssetVec[i] = (AcqAsset + TarAsset)/deflator[Year]

            AcqDep = EasyData2016[EasyData2016["Cert/ID"] == AcqID]["Total Domestic Deposits"].to_numpy()
            TarDep = EasyData2016[EasyData2016["Cert/ID"] == TarID]["Total Domestic Deposits"].to_numpy()

            if len(AcqDep) == 0  or len(TarDep) == 0:
                DepositVec[i] = None
            else:
                DepositVec[i] = (AcqDep + TarDep)/deflator[Year]


        
        

    return AssetVec,DepositVec

A deflator is used to deflate the monetary amounts. This comes from the FED.

In [81]:
deflator = dict()

deflator[2012] = 1
deflator[2013] = 1.01
deflator[2014] = 1.03
deflator[2015] = 1.03
deflator[2016] = 1.05
deflator[2017] = 1.07
deflator[2018] = 1.09
deflator[2019] = 1.11


The function is run on the obtained sample of M&A transactions and the results are appended to the list.

In [82]:
x,y = getMAAssetsAndDeposits(MATrans1417,deflator)

MATrans1417["Total Assets Prev Year"] = x
MATrans1417["Total Deposits Prev Year"] = y

In [83]:
MATrans1417Sorted = MATrans1417.sort_values("Total Assets Prev Year",ascending = False).dropna().reset_index(drop = True) # M&A Transactions are sorted by Total Assets Previous Year

This sample contains firms that have participated in multiple M&A transactions ina  +- 2 year window. These are excluded by using the following function to get the good indices:

In [84]:
def getTopXIndices(N,MATrans1417Sorted,MATrans1219):
    '''This Function returns the top N M&A Transactions by excluding companies that did multiple deals'''
    nGoodMA = 0
    i = 0

    # List of Good indices
    iGood = []


    while nGoodMA < N:
        # Obtain Acquirer and Target ID
        AcqID = MATrans1417Sorted["ACQ_CERT"][i]
        TarID = MATrans1417Sorted["OUT_CERT"][i]
        Year = MATrans1417Sorted["EFFYEAR"][i]

        # Create a copy of the extended DataFrame
        MATrans1219_ = MATrans1219[MATrans1219["EFFYEAR"]>= Year - 2]
        MATrans1219_ = MATrans1219_[MATrans1219_["EFFYEAR"]<= Year + 2]
        
        # Create list of all IDs in this window
        IDsWindow = np.hstack([MATrans1219_["ACQ_CERT"].to_numpy(),MATrans1219_["OUT_CERT"].to_numpy()])
        
        # Find positions of current Deal in this list
        OccurenceAcqID = np.where(IDsWindow == AcqID)[0]
        OccurenceTarID = np.where(IDsWindow == TarID)[0]

        # Check whether the a bank is part of more than one deal (we know it must be part of at least one)
        if OccurenceAcqID.shape[0] == 1 and OccurenceTarID.shape[0] == 1:

            nGoodMA += 1
            iGood.append(i)
        i += 1

    return iGood

In [85]:
indices = getTopXIndices(50,MATrans1417Sorted,MATrans1219) # Get Indices
MAPreCensor = MATrans1417Sorted.iloc[indices,:].reset_index(drop=True) # Get rid of all other M&A transactions


We get rid of two Deals: for i = 28 (First NBC Bank), there is no data in the whole observational period, and Southside Bank (i = 21) had two Deals (outside the windows), we exclude this to ensure a DMU is not compared to itself in the same time period.

In [90]:
# Rename Columns
MAPairs = MA[["ACQ_CERT","ACQ_INSTNAME","OUT_CERT","OUT_INSTNAME","EFFYEAR"]].rename(columns = {"ACQ_CERT":"ACQ","OUT_CERT":"TAR","EFFYEAR":"YEAR","ACQ_INSTNAME": "ACQ_Name","OUT_INSTNAME":"TAR_Name"})

# Drop two DMUs for reasons explained above
MAPairs.drop([28,21],inplace = True) # i = 28 closed before the end of the observation period

MAPairs = MAPairs.reset_index(drop=True)


The following Banks exhibit zeros in their data and need to be excluded. This observation was made at a later stage, but to ensure consistency of this notebok it is presented here:

In [87]:
BadDMU = [628,34775,21843]

In [91]:
MAPairs = MAPairs.loc[~MAPairs["ACQ"].isin(BadDMU)].reset_index(drop = True) # Drop Deals with zeros in Data

In [89]:
MAPairs

,ACQ,ACQ_Name,TAR,TAR_Name,YEAR
0,18409,"TD Bank, National Association",58656,Scottrade Bank,2017
1,588,Manufacturers and Traders Trust Company,13074,Hudson City Savings Bank,2015
2,58978,"CIT BANK, NATIONAL ASSOCIATION",35575,CIT Bank,2015
3,5510,Frost Bank,22482,Western National Bank,2014
4,12441,Hancock Bank,33029,Whitney Bank,2014
5,4988,Trustmark National Bank,34866,Reliance Bank,2017
6,26610,Bank of Hope,23301,Wilshire Bank,2016
7,15289,Great Western Bank,28092,Home Federal Bank,2016
8,17491,PlainsCapital Bank,32460,"Southwest Securities , FSB",2015
9,28178,Northwest Bank,14832,The Lorain National Bank,2015


Note that the names amy not match the ones in the Dissertation as they have been edited for clarity. One may check the FDIC bank finder for questions regarding the naming of the banks in the DIssertation.

## 2. Obtain and clean the data for the Banks in the sample